In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf

from glob import glob

In [20]:
from platform import python_version

print(python_version())

3.8.8


In [7]:
IMAGE_SIZE = [224, 224]

train_path = 'D:/Lungs_Image_Data/DATA'

In [8]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

InceptionResNetV2_1 = InceptionResNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [9]:
for layer in InceptionResNetV2_1.layers:
    layer.trainable = False

In [11]:
flattened = tf.keras.layers.Flatten()(InceptionResNetV2_1.output)
fc1 = tf.keras.layers.Dense(256, activation='relu', name="AddedDense1")(flattened)
d = tf.keras.layers.Dropout(0.25)(fc1)
fc2 = tf.keras.layers.Dense(128, activation='relu', name="AddedDense2")(d)
d2=tf.keras.layers.Dropout(0.25)(fc2)
fc3 = tf.keras.layers.Dense(6, activation='softmax', name="AddedDense3")(d2)
model = tf.keras.models.Model(inputs=InceptionResNetV2_1.input, outputs=fc3)



In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 111, 111, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 111, 111, 32) 0           batch_normalization_203[0][0]    
______________________________________________________________________________________________

In [13]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='nadam',
  metrics=['accuracy']
)

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,validation_split=0.2)


In [15]:
training_set = train_datagen.flow_from_directory(directory='D:/Lungs_Image_Data/DATA',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 shuffle=True,
                                                 class_mode = 'categorical',
                                                 subset='training')
                              

Found 25364 images belonging to 6 classes.


In [16]:
test_set = train_datagen.flow_from_directory(directory='D:/Lungs_Image_Data/DATA',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            shuffle=True,
                                            class_mode = 'categorical',
                                            subset= "validation")

Found 6339 images belonging to 6 classes.


In [17]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  validation_steps = int(len(test_set)/64)
  
)


Epoch 1/20
397/397 [==============================] - 598s 1s/step - loss: 1.0960 - accuracy: 0.7642 - val_loss: 0.7865 - val_accuracy: 0.7500
Epoch 2/20
397/397 [==============================] - 296s 745ms/step - loss: 0.4588 - accuracy: 0.8447 - val_loss: 0.5263 - val_accuracy: 0.8750
Epoch 3/20
397/397 [==============================] - 295s 744ms/step - loss: 0.3778 - accuracy: 0.8704 - val_loss: 0.6904 - val_accuracy: 0.7656
Epoch 4/20
397/397 [==============================] - 297s 747ms/step - loss: 0.3453 - accuracy: 0.8804 - val_loss: 0.7535 - val_accuracy: 0.7031
Epoch 5/20
397/397 [==============================] - 290s 730ms/step - loss: 0.3181 - accuracy: 0.8873 - val_loss: 0.8945 - val_accuracy: 0.7500
Epoch 6/20
397/397 [==============================] - 287s 722ms/step - loss: 0.2907 - accuracy: 0.8971 - val_loss: 0.5856 - val_accuracy: 0.7812
Epoch 7/20
397/397 [==============================] - 283s 712ms/step - loss: 0.2750 - accuracy: 0.9021 - val_loss: 0.5785 - va

In [21]:
from tensorflow.keras.models import load_model

model.save('Inceptionresnet_lungs_e20_acc92_val_90.pkl')

INFO:tensorflow:Assets written to: Inceptionresnet_lungs_e20_acc92_val_90.pkl\assets


In [4]:
from keras.models import load_model
model = load_model('C:\multi disease\Inceptionresnet_lungs_e20_acc92_val_90.h5')

In [ ]:
img_width, img_height = 224,224

Error: Session cannot generate requests